# E1: Four-Layer Cascade with Non-Decreasing Coherence

## RTM Cascade Framework - Signature S1 Validation

This notebook validates that the coherence exponent α increases (or does not decrease) across nested layers in a cascade architecture.

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

# Parameters
N_LAYERS = 4
ALPHA_BASE = 2.0
DELTA_ALPHA = 0.3
L_MIN, L_MAX = 10, 200
N_SIZES = 10
N_EVENTS = 50
NOISE_SIGMA = 0.15
N_BOOTSTRAP = 1000
EPSILON_TOL = 0.05
LAYER_FACTORS = [1.0, 2.5, 0.8, 3.2]
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)
print("Parameters loaded.")

## 1. Generate Cascade Data

Model: $T_n(L) = c_n \cdot L^{\alpha_n} \cdot \varepsilon$ where $\alpha_n = \alpha_0 + n \cdot \Delta\alpha$

In [ ]:
L_values = np.geomspace(L_MIN, L_MAX, N_SIZES)
records = []

for n in range(N_LAYERS):
    alpha_n = ALPHA_BASE + n * DELTA_ALPHA
    c_n = LAYER_FACTORS[n]
    
    for L in L_values:
        for _ in range(N_EVENTS):
            epsilon = rng.lognormal(0, NOISE_SIGMA)
            T = c_n * (L ** alpha_n) * epsilon
            records.append({
                'layer': n, 'L': L, 'T': T,
                'log_L': np.log10(L), 'log_T': np.log10(T),
                'alpha_true': alpha_n, 'c_true': c_n
            })

df = pd.DataFrame(records)
print(f"Generated {len(df)} observations across {N_LAYERS} layers")
df.head()

## 2. Estimate α per Layer

In [ ]:
def bootstrap_slope(log_L, log_T, n_boot=1000):
    slopes = []
    for _ in range(n_boot):
        idx = rng.choice(len(log_L), len(log_L), replace=True)
        slope, _, _, _, _ = stats.linregress(log_L[idx], log_T[idx])
        slopes.append(slope)
    return np.mean(slopes), np.percentile(slopes, 2.5), np.percentile(slopes, 97.5)

layer_results = []
for n in range(N_LAYERS):
    layer_data = df[df['layer'] == n]
    log_L = layer_data['log_L'].values
    log_T = layer_data['log_T'].values
    
    slope, intercept, r_value, _, _ = stats.linregress(log_L, log_T)
    alpha_est, ci_lo, ci_hi = bootstrap_slope(log_L, log_T, N_BOOTSTRAP)
    
    layer_results.append({
        'layer': n,
        'alpha_true': layer_data['alpha_true'].iloc[0],
        'alpha_est': alpha_est,
        'ci_lower': ci_lo,
        'ci_upper': ci_hi,
        'R2': r_value**2
    })
    print(f"Layer {n}: α = {alpha_est:.4f} [{ci_lo:.4f}, {ci_hi:.4f}] (true: {layer_data['alpha_true'].iloc[0]:.2f})")

results_df = pd.DataFrame(layer_results)
results_df

## 3. Test S1: Monotone Coherence

In [ ]:
print("Testing S1: Non-decreasing α across layers\n")
all_pass = True

for i in range(N_LAYERS - 1):
    delta = layer_results[i+1]['alpha_est'] - layer_results[i]['alpha_est']
    ci_w1 = (layer_results[i]['ci_upper'] - layer_results[i]['ci_lower']) / 2
    ci_w2 = (layer_results[i+1]['ci_upper'] - layer_results[i+1]['ci_lower']) / 2
    delta_ci = np.sqrt(ci_w1**2 + ci_w2**2) * 1.96
    
    passes = (delta - delta_ci) > -EPSILON_TOL
    all_pass = all_pass and passes
    status = "✓ PASS" if passes else "✗ FAIL"
    print(f"Δα({i}→{i+1}) = {delta:.4f} CI: [{delta-delta_ci:.4f}, {delta+delta_ci:.4f}] {status}")

print(f"\nS1 Overall: {'PASS ✓' if all_pass else 'FAIL ✗'}")

## 4. Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, N_LAYERS))

# Left: Log-log scaling
for n in range(N_LAYERS):
    layer_data = df[df['layer'] == n]
    axes[0].scatter(layer_data['log_L'], layer_data['log_T'], 
                    alpha=0.3, s=10, color=colors[n], label=f'Layer {n}')

axes[0].set_xlabel('log₁₀(L)', fontsize=12)
axes[0].set_ylabel('log₁₀(T)', fontsize=12)
axes[0].set_title('RTM Scaling: T ∝ L^α per Layer', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: α estimates with CI
layers = [r['layer'] for r in layer_results]
alphas = [r['alpha_est'] for r in layer_results]
yerr_lo = [r['alpha_est'] - r['ci_lower'] for r in layer_results]
yerr_hi = [r['ci_upper'] - r['alpha_est'] for r in layer_results]
alpha_true = [r['alpha_true'] for r in layer_results]

axes[1].errorbar(layers, alphas, yerr=[yerr_lo, yerr_hi],
                 fmt='o', markersize=12, capsize=8, color='blue', label='Estimated α')
axes[1].plot(layers, alpha_true, 's--', markersize=10, color='red', label='True α')
axes[1].set_xlabel('Layer n', fontsize=12)
axes[1].set_ylabel('Coherence Exponent α', fontsize=12)
axes[1].set_title('S1: Monotone Coherence Across Layers', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('output/E1_notebook_plot.png', dpi=150)
plt.show()

## 5. Conclusion

The simulation confirms **Signature S1**: coherence (α) increases monotonically across layers.

- Layer-level factors (c_n) affect intercepts only, not slopes
- The RTM cascade hypothesis is supported by the data